<a href="https://colab.research.google.com/github/pszemraj/ml4hc-s22-project01/blob/autogluon-results/notebooks/colab/automl-baseline/process_autogluon_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#process_autogluon_results

- cleans up the dataframes a bit for the report


# setup

In [1]:
#@markdown add auto-Colab formatting with `IPython.display`
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
!pip install -U plotly orca kaleido -q
import plotly.express as px

     |████████████████████████████████| 27.7 MB 76.4 MB/s 
     |████████████████████████████████| 244 kB 36.1 MB/s 
     |████████████████████████████████| 79.9 MB 88 kB/s 
     |████████████████████████████████| 4.3 MB 44.5 MB/s 


In [4]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os

In [5]:
#@title mount drive
from google.colab import drive

drive_base_str = '/content/drive'
drive.mount(drive_base_str)


Mounted at /content/drive


In [6]:
#@markdown determine root
import os
from pathlib import Path
peter_base = Path('/content/drive/MyDrive/ETHZ-2022-S/ML-healthcare-projects/project1/gluon-autoML/')

if peter_base.exists() and peter_base.is_dir():
    path = str(peter_base.resolve())
else:
    # original
    path = '/content/drive/MyDrive/ETH/'

print(f"base drive dir is:\n{path}")

base drive dir is:
/content/drive/MyDrive/ETHZ-2022-S/ML-healthcare-projects/project1/gluon-autoML


## define folder for outputs

In [7]:
_out_dir_name = "Formatted-results-report" #@param {type:"string"}

output_path = os.path.join(path, _out_dir_name)
os.makedirs(output_path, exist_ok=True)
print(f"notebook outputs will be stored in:\n{output_path}")

notebook outputs will be stored in:
/content/drive/MyDrive/ETHZ-2022-S/ML-healthcare-projects/project1/gluon-autoML/Formatted-results-report


In [8]:
_out = Path(output_path)
_src = Path(path)

##load data

### MIT

In [10]:
data_dir = _src / "final-results"

csv_files = {f.stem:f for f in data_dir.iterdir() if f.is_file() and f.suffix=='.csv'}

print(csv_files)

{'mitbih_autogluon_results': PosixPath('/content/drive/MyDrive/ETHZ-2022-S/ML-healthcare-projects/project1/gluon-autoML/final-results/mitbih_autogluon_results.csv'), 'ptbdb_autogluon_results': PosixPath('/content/drive/MyDrive/ETHZ-2022-S/ML-healthcare-projects/project1/gluon-autoML/final-results/ptbdb_autogluon_results.csv')}


In [11]:
mit_ag = pd.read_csv(csv_files['mitbih_autogluon_results'])
mit_ag.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               20 non-null     int64  
 1   model                    20 non-null     object 
 2   score_test               20 non-null     float64
 3   score_val                20 non-null     float64
 4   pred_time_test           20 non-null     float64
 5   pred_time_val            20 non-null     float64
 6   fit_time                 20 non-null     float64
 7   pred_time_test_marginal  20 non-null     float64
 8   pred_time_val_marginal   20 non-null     float64
 9   fit_time_marginal        20 non-null     float64
 10  stack_level              20 non-null     int64  
 11  can_infer                20 non-null     bool   
 12  fit_order                20 non-null     int64  
dtypes: bool(1), float64(8), int64(3), object(1)
memory usage: 2.0+ KB


In [13]:
mit_ag.sort_values(by='score_val', ascending=False, inplace=True)
mit_ag.head()

,Unnamed: 0,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
2,2,WeightedEnsemble_L3,0.987164,0.987756,389.968791,790.527368,8732.493713,0.012632,0.007183,12.085104,3,True,20
3,3,LightGBM_BAG_L2,0.987073,0.987505,365.506774,736.025195,6830.926788,2.180315,1.786115,259.455806,2,True,11
7,7,XGBoost_BAG_L2,0.986479,0.987493,365.984499,736.120318,6614.919820,2.658040,1.881239,43.448839,2,True,17
4,4,NeuralNetFastAI_BAG_L2,0.987027,0.987471,368.599604,738.069256,7185.616762,5.273145,3.830176,614.145781,2,True,9
1,1,RandomForestEntr_BAG_L2,0.987256,0.987402,364.258731,746.033134,6843.372016,0.932272,11.794054,271.901035,2,True,13


In [14]:
orig_cols = list(mit_ag.columns)
new_cols = []
for i, col in enumerate(orig_cols):
    col = col.lower()
    if 'unnamed' in col:
        new_cols.append(f"delete_me_{i}")
        continue
    col = col.replace('score', 'accuracy')
    new_cols.append(col)

mit_ag.columns = new_cols

In [15]:
mit_ag.columns

Index(['delete_me_0', 'model', 'accuracy_test', 'accuracy_val',
       'pred_time_test', 'pred_time_val', 'fit_time',
       'pred_time_test_marginal', 'pred_time_val_marginal',
       'fit_time_marginal', 'stack_level', 'can_infer', 'fit_order'],
      dtype='object')

In [17]:
try:
    del mit_ag['delete_me_0']
except Exception as e:
    print(f'skipping delete - {e}')

In [19]:
mit_ag.reset_index(drop=True, inplace=True)
mit_ag.head()

,model,accuracy_test,accuracy_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.987164,0.987756,389.968791,790.527368,8732.493713,0.012632,0.007183,12.085104,3,True,20
1,LightGBM_BAG_L2,0.987073,0.987505,365.506774,736.025195,6830.926788,2.180315,1.786115,259.455806,2,True,11
2,XGBoost_BAG_L2,0.986479,0.987493,365.984499,736.120318,6614.919820,2.658040,1.881239,43.448839,2,True,17
3,NeuralNetFastAI_BAG_L2,0.987027,0.987471,368.599604,738.069256,7185.616762,5.273145,3.830176,614.145781,2,True,9
4,RandomForestEntr_BAG_L2,0.987256,0.987402,364.258731,746.033134,6843.372016,0.932272,11.794054,271.901035,2,True,13


#### save mit-gluon-reformat

In [25]:
mit_ag.to_csv(_out / "MITBIH_autogluon_baseline_results_Accuracy.csv", index=False)

## PTB reformat

In [21]:
ptb_ag = pd.read_csv(csv_files['ptbdb_autogluon_results']).convert_dtypes()
ptb_ag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               26 non-null     Int64  
 1   model                    26 non-null     string 
 2   score_test               26 non-null     Float64
 3   score_val                26 non-null     Float64
 4   pred_time_test           26 non-null     Float64
 5   pred_time_val            26 non-null     Float64
 6   fit_time                 26 non-null     Float64
 7   pred_time_test_marginal  26 non-null     Float64
 8   pred_time_val_marginal   26 non-null     Float64
 9   fit_time_marginal        26 non-null     Float64
 10  stack_level              26 non-null     Int64  
 11  can_infer                26 non-null     boolean
 12  fit_order                26 non-null     Int64  
dtypes: Float64(8), Int64(3), boolean(1), string(1)
memory usage: 2.9 KB


In [22]:
ptb_ag.sort_values(by='score_val', ascending=False, inplace=True)
ptb_ag.head()

,Unnamed: 0,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
21,21,WeightedEnsemble_L3,1.0,0.991153,56.442027,55.951196,7574.346129,0.0027,0.017853,4.621803,3,True,26
17,17,LightGBM_BAG_L2,1.0,0.990981,53.88617,54.051606,6467.99865,1.216053,1.016706,452.150008,2,True,16
22,22,NeuralNetFastAI_BAG_L2,1.0,0.990981,59.402995,60.666399,6712.757208,6.732878,7.631498,696.908567,2,True,22
20,20,LightGBMLarge_BAG_L2,1.0,0.990637,55.223274,54.916638,7117.574319,2.553157,1.881737,1101.725677,2,True,25
11,11,WeightedEnsemble_L2,1.0,0.98995,22.309041,19.853544,2162.411832,0.007025,0.017502,5.212323,2,True,14


In [23]:
orig_cols = list(ptb_ag.columns)
new_cols = []
for i, col in enumerate(orig_cols):
    col = col.lower()
    if 'unnamed' in col:
        new_cols.append(f"delete_me_{i}")
        continue
    col = col.replace('score', 'roc_auc')
    new_cols.append(col)

ptb_ag.columns = new_cols
print(f'the columns for the ptb results are now:\n{ptb_ag.columns}')

the columns for the ptb results are now:
Index(['delete_me_0', 'model', 'roc_auc_test', 'roc_auc_val', 'pred_time_test',
       'pred_time_val', 'fit_time', 'pred_time_test_marginal',
       'pred_time_val_marginal', 'fit_time_marginal', 'stack_level',
       'can_infer', 'fit_order'],
      dtype='object')


In [24]:
try:
    del ptb_ag['delete_me_0']
except Exception as e:
    print(f'skipping delete - {e}')


ptb_ag.reset_index(drop=True, inplace=True)
ptb_ag.head()

,model,roc_auc_test,roc_auc_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,1.0,0.991153,56.442027,55.951196,7574.346129,0.0027,0.017853,4.621803,3,True,26
1,LightGBM_BAG_L2,1.0,0.990981,53.88617,54.051606,6467.99865,1.216053,1.016706,452.150008,2,True,16
2,NeuralNetFastAI_BAG_L2,1.0,0.990981,59.402995,60.666399,6712.757208,6.732878,7.631498,696.908567,2,True,22
3,LightGBMLarge_BAG_L2,1.0,0.990637,55.223274,54.916638,7117.574319,2.553157,1.881737,1101.725677,2,True,25
4,WeightedEnsemble_L2,1.0,0.98995,22.309041,19.853544,2162.411832,0.007025,0.017502,5.212323,2,True,14


In [26]:
ptb_ag.to_csv(_out / "PTBDB_autogluon_baseline_results_ROCAUC.csv", index=False)


In [27]:
print(f'results are in {_out.resolve()}')

results are in /content/drive/MyDrive/ETHZ-2022-S/ML-healthcare-projects/project1/gluon-autoML/Formatted-results-report
